In [ ]:
%pwd

'd:\\End-to-End-Medical-Chatbot\\research'

In [1]:
import os
os.chdir("../")
%pwd

'd:\\End-to-End-Medical-Chatbot'

In [61]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import DirectoryLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [62]:
def load_pdf_files(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls = PyPDFLoader)
    
    documents= loader.load()
    return documents


In [63]:
extracted_data = load_pdf_files("data/")

In [64]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [65]:
text_chunks = text_split(extracted_data)
print(len(text_chunks))

7023


In [66]:
from langchain_huggingface import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [67]:
embeddings = download_hugging_face_embeddings()

In [68]:
query = embeddings.embed_query("Hello, world!")
print(len(query))

384


In [69]:
from dotenv import load_dotenv
load_dotenv()
import os

In [70]:
api_key = os.environ["PINECONE_API_KEY"] 
api_key

'pcsk_4fZ6Gq_8WEpqn6eqAiLSsPCq2e7o3uEScP8pY5MdYXQFXBHX2SgWAh4SH6A7QLLNauLpKF'

In [43]:
from pinecone import Pinecone ,ServerlessSpec
pc = Pinecone(api_key= api_key )

In [71]:
# Import the Pinecone library
from pinecone import Pinecone ,ServerlessSpec


# Initialize a Pinecone client with your API key
pc = Pinecone(api_key= api_key )

# Create a dense index with integrated embedding
index_name = "medicalchatbot"

pc.create_index(
    name = index_name,
    dimension=384,
    metric="cosine",
    spec= ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


In [75]:
from langchain_pinecone import PineconeVectorStore

index_name = "medicalchatbot"
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)


In [76]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})


In [77]:
retrieved_docs = retriever.invoke("What is Acne ?")

In [50]:
retrieved_docs

[Document(id='9ef692c6-603d-43ed-b12b-73b40738bc11', metadata={'creationdate': '2004-12-18T17:16:32-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:35:04-06:00', 'page': 425.0, 'page_label': '426', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Gale Encyclopedia of Medicine. Vol. 2. 2nd ed.pdf', 'total_pages': 759.0}, page_content='contact has sensitized the body.\nCorticosteriod —A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied to the skin.\nAfter 24–48 hours, if the area is red and swollen,\nthe test is positive for that substance. If no reaction\noccurs, another substance is applied. This is con-'),
 Document(id='ac48c3b2-7776-4292-a636-7a5c63cf3fd8', metadata={'creationdate': '2004-12-18T17:16:32-05:00', 'creator': 'PyPDF', 'moddate': '

In [78]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama3.2",temperature=0.4)


In [79]:
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


In [80]:


system_prompt = (
    "You are an assistant for question-answering task."
    "Use he following pieces of retrieved context to answer"
    "the question. If you don't know the answer , say that you don't know."
    "Use three sentence maximum and keep the answer concise."
    "\n\n"
    "{context}"

)

In [81]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [82]:
question_and_answer = create_stuff_documents_chain(llm,prompt)

rag_chain = create_retrieval_chain(retriever,question_and_answer)

In [84]:
response = rag_chain.invoke({"input":"What is Acne ?"})
print(response["answer"])

Acne is a skin condition characterized by inflammation, redness, and comedones (blackheads or whiteheads) caused by the clogged pores and bacterial growth. It affects millions of people worldwide, particularly during puberty, adolescence, and early adulthood. Acne can be treated with various methods, including topical treatments, antibiotics, and hormonal therapies.


In [85]:
response = rag_chain.invoke({"input":"What is Statistics"})
print(response["answer"])

Statistics are used to predict the future course and outcome of a disease, such as cancer. Five-year survival rates measure the proportion of people with cancer expected to be alive five years after diagnosis compared to a similar population without cancer. These statistics provide important factors for doctors to assess the likelihood of recovery.


In [86]:
response = rag_chain.invoke({"input":"What is Stats"})
print(response["answer"])

MaxRetryError: HTTPSConnectionPool(host='medicalchatbot-ad71n6o.svc.aped-4627-b74a.pinecone.io', port=443): Max retries exceeded with url: /query (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000259E75A8320>: Failed to resolve 'medicalchatbot-ad71n6o.svc.aped-4627-b74a.pinecone.io' ([Errno 11001] getaddrinfo failed)"))